# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path

import numpy as np
import pandas as pd
import sys

In [ ]:
#| hide
from nbdev import show_doc

In [ ]:
#| export
def display_mds(*strings:str # any number of strings with text in markdown format
               ):
    """Utility function to display several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
           )

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display several DataFrame at once"""
    for df in dfs:
        display(df)

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,-0.457386,-1.809706,0.013889,-1.162611,-0.386687
1,0.873220,0.834767,-0.714602,-0.361059,0.314682
2,-0.324666,-2.471258,0.083272,1.109136,0.666768


,0,1,2,3,4,5,6,7,8,9
0,-0.172308,-0.279366,0.140218,1.414217,-0.592052,0.384194,-0.339798,-0.472517,-0.914254,-0.071302
1,-0.011717,-0.012469,0.785173,-0.129211,1.075478,-0.514928,1.233203,0.762092,-0.029894,2.816166
2,-0.370095,0.165443,0.292777,-0.792470,0.178790,-0.020105,0.231050,-0.097163,1.036050,0.699237


In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of nb for setting up autoreload, paths, ... """
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def colab_install_project_code(
    package_name:str # project package name, e.g. git+https://github.com/vtecftwy/metagentools.git@main
):
    """When nb running on colab, pip install the project code package"""
    try:
        from google.colab import drive
        ON_COLAB = True
        print('The notebook is running on colab')
        print('Installing project code')
        cmd = f"pip install -U {package_name}"
        run(cmd)

    except ModuleNotFoundError:
        ON_COLAB = False
        print('The notebook is running locally, will not automatically install project code')

    return ON_COLAB

When using colab, the project code must be install every time from its github repo or from the Python Package Index (PyPI).

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
colab_install_project_code(package_name='git+https://github.com/vtecftwy/metagentools.git@main')

The notebook is running locally, will not automatically install project code


False